In [30]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
import numpy as np
import cv2
import os

In [13]:
train_dir = "C:/Users/Lohith/Desktop/Captsone Review 2/frame_of_shots/frame_of_shots"
validation_dir = "C:/Users/Lohith/Desktop/Captsone Review 2/frame_of_shots/frame_of_shots"

In [14]:
train_datagen = ImageDataGenerator(
    rescale = 1./255.,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

In [15]:
test_datagen = ImageDataGenerator(rescale = 1.0/255.)

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224))

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (224, 224))

Found 10000 images belonging to 2 classes.
Found 10000 images belonging to 2 classes.


In [17]:
base_model = VGG16(
    input_shape = (224, 224, 3),
    include_top = False,
    weights = 'imagenet')

In [18]:
for layer in base_model.layers:
    layer.trainable = False

In [19]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.models.Model(base_model.input, x)

In [20]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [21]:
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
    loss = 'binary_crossentropy',
    metrics = ['acc'])

In [22]:
from datetime import datetime
start_time = datetime.now()
try:
    vgghist = model.fit(
        train_generator,
        validation_data = validation_generator,
        steps_per_epoch = 3,
        epochs = 5)
except:
    pass
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Epoch 1/5
3/3 [==============================] - 1533s 764s/step - loss: 1.5761 - acc: 0.5833 - val_loss: 0.8537 - val_acc: 0.5227
Epoch 2/5
3/3 [==============================] - 1615s 806s/step - loss: 1.4307 - acc: 0.4000 - val_loss: 0.9015 - val_acc: 0.5000
Epoch 3/5
3/3 [==============================] - 1473s 734s/step - loss: 0.5546 - acc: 0.7500 - val_loss: 0.3153 - val_acc: 0.8862
Epoch 4/5
3/3 [==============================] - 1580s 788s/step - loss: 0.5983 - acc: 0.6500 - val_loss: 0.5498 - val_acc: 0.7225
Epoch 5/5
3/3 [==============================] - 1499s 747s/step - loss: 0.6101 - acc: 0.6167 - val_loss: 0.2730 - val_acc: 0.9016
Duration: 2:08:20.761217


In [23]:
model.save('vgg1.h5')

In [39]:
from tensorflow.keras.models import load_model

In [40]:
import cv2
path="C://Users//Lohith//Downloads//Untitled video - Made with Clipchamp (5).mp4"
cap = cv2.VideoCapture("C://Users//Lohith//Downloads//Untitled video - Made with Clipchamp (5).mp4")
no_of_frame = int(cap.get(7))

In [41]:
import numpy as np
import cv2
temp = []
f = open("Pitch and non-pitch classification.txt", "w")
f.write("Video Path : "+path+"\n")
f.write("\n"+"Overall Training time taken by the shot classification model : 2:08:20.761217 "+"\n")
f.write("\n"+"Time Analysis for shot classification on each frame"+"\n")
from datetime import datetime
start_time = datetime.now()
z=2
i = 0
while i < no_of_frame:
    ret,frame = cap.read()
    frame = cv2.resize(frame,(224,224))
    img = np.asarray(frame)
    img = np.expand_dims(img,axis=0)
    saved_model = load_model("C:/Users/Lohith/vgg1.h5")
    output = saved_model.predict(img)
    val = sum(output[0])
    if val < 0.95:
        temp.append('Non-Pitch')
    else:
        temp.append('Pitch')
    print(str(round((i)/(no_of_frame)*100,2))+'% completed!')
    end_time = datetime.now()
    if i//30==z:
        f.write("Time taken for "+str(z)+" seconds"+"\n")
        f.write('Duration: {}'.format(end_time - start_time))
        f.write("\n")
        z=z*2
    i+=1

print(temp)

1/1 [==============================] - 0s 293ms/step
0.0% completed!
1/1 [==============================] - 0s 306ms/step
0.1% completed!
1/1 [==============================] - 1s 852ms/step
0.2% completed!
1/1 [==============================] - 1s 550ms/step
0.3% completed!
1/1 [==============================] - 0s 363ms/step
0.4% completed!
1/1 [==============================] - 0s 305ms/step
0.5% completed!
1/1 [==============================] - 0s 411ms/step
0.6% completed!
1/1 [==============================] - 1s 539ms/step
0.7% completed!
1/1 [==============================] - 0s 400ms/step
0.8% completed!
1/1 [==============================] - 0s 282ms/step
0.9% completed!
1/1 [==============================] - 0s 298ms/step
1.0% completed!
1/1 [==============================] - 0s 383ms/step
1.1% completed!
1/1 [==============================] - 0s 372ms/step
1.2% completed!
1/1 [==============================] - 1s 755ms/step
1.3% completed!
1/1 [=============================

1/1 [==============================] - 0s 295ms/step
11.69% completed!
1/1 [==============================] - 0s 281ms/step
11.79% completed!
1/1 [==============================] - 0s 282ms/step
11.89% completed!
1/1 [==============================] - 0s 380ms/step
11.99% completed!
1/1 [==============================] - 0s 408ms/step
12.09% completed!
1/1 [==============================] - 0s 269ms/step
12.19% completed!
1/1 [==============================] - 0s 372ms/step
12.29% completed!
1/1 [==============================] - 0s 355ms/step
12.39% completed!
1/1 [==============================] - 0s 325ms/step
12.49% completed!
1/1 [==============================] - 0s 268ms/step
12.59% completed!
1/1 [==============================] - 0s 235ms/step
12.69% completed!
1/1 [==============================] - 0s 267ms/step
12.79% completed!
1/1 [==============================] - 0s 266ms/step
12.89% completed!
1/1 [==============================] - 0s 282ms/step
12.99% completed!
1/1 [=

1/1 [==============================] - 0s 282ms/step
23.18% completed!
1/1 [==============================] - 0s 298ms/step
23.28% completed!
1/1 [==============================] - 0s 351ms/step
23.38% completed!
1/1 [==============================] - 0s 409ms/step
23.48% completed!
1/1 [==============================] - 1s 555ms/step
23.58% completed!
1/1 [==============================] - 0s 440ms/step
23.68% completed!
1/1 [==============================] - 0s 439ms/step
23.78% completed!
1/1 [==============================] - 0s 258ms/step
23.88% completed!
1/1 [==============================] - 0s 251ms/step
23.98% completed!
1/1 [==============================] - 0s 268ms/step
24.08% completed!
1/1 [==============================] - 0s 246ms/step
24.18% completed!
1/1 [==============================] - 0s 251ms/step
24.28% completed!
1/1 [==============================] - 0s 252ms/step
24.38% completed!
1/1 [==============================] - 0s 423ms/step
24.48% completed!
1/1 [=

1/1 [==============================] - 0s 472ms/step
34.67% completed!
1/1 [==============================] - 0s 267ms/step
34.77% completed!
1/1 [==============================] - 0s 377ms/step
34.87% completed!
1/1 [==============================] - 0s 314ms/step
34.97% completed!
1/1 [==============================] - 0s 268ms/step
35.06% completed!
1/1 [==============================] - 0s 251ms/step
35.16% completed!
1/1 [==============================] - 0s 266ms/step
35.26% completed!
1/1 [==============================] - 0s 251ms/step
35.36% completed!
1/1 [==============================] - 0s 267ms/step
35.46% completed!
1/1 [==============================] - 0s 246ms/step
35.56% completed!
1/1 [==============================] - 0s 252ms/step
35.66% completed!
1/1 [==============================] - 0s 267ms/step
35.76% completed!
1/1 [==============================] - 0s 266ms/step
35.86% completed!
1/1 [==============================] - 0s 279ms/step
35.96% completed!
1/1 [=

1/1 [==============================] - 0s 267ms/step
46.15% completed!
1/1 [==============================] - 1s 551ms/step
46.25% completed!
1/1 [==============================] - 0s 458ms/step
46.35% completed!
1/1 [==============================] - 0s 299ms/step
46.45% completed!
1/1 [==============================] - 0s 275ms/step
46.55% completed!
1/1 [==============================] - 0s 271ms/step
46.65% completed!
1/1 [==============================] - 0s 251ms/step
46.75% completed!
1/1 [==============================] - 0s 262ms/step
46.85% completed!
1/1 [==============================] - 0s 267ms/step
46.95% completed!
1/1 [==============================] - 0s 276ms/step
47.05% completed!
1/1 [==============================] - 0s 266ms/step
47.15% completed!
1/1 [==============================] - 0s 258ms/step
47.25% completed!
1/1 [==============================] - 0s 330ms/step
47.35% completed!
1/1 [==============================] - 0s 251ms/step
47.45% completed!
1/1 [=

1/1 [==============================] - 0s 250ms/step
57.64% completed!
1/1 [==============================] - 0s 362ms/step
57.74% completed!
1/1 [==============================] - 0s 284ms/step
57.84% completed!
1/1 [==============================] - 0s 240ms/step
57.94% completed!
1/1 [==============================] - 0s 242ms/step
58.04% completed!
1/1 [==============================] - 0s 267ms/step
58.14% completed!
1/1 [==============================] - 0s 266ms/step
58.24% completed!
1/1 [==============================] - 0s 269ms/step
58.34% completed!
1/1 [==============================] - 0s 360ms/step
58.44% completed!
1/1 [==============================] - 0s 266ms/step
58.54% completed!
1/1 [==============================] - 0s 251ms/step
58.64% completed!
1/1 [==============================] - 0s 251ms/step
58.74% completed!
1/1 [==============================] - 0s 241ms/step
58.84% completed!
1/1 [==============================] - 0s 268ms/step
58.94% completed!
1/1 [=

1/1 [==============================] - 0s 296ms/step
69.13% completed!
1/1 [==============================] - 0s 262ms/step
69.23% completed!
1/1 [==============================] - 0s 258ms/step
69.33% completed!
1/1 [==============================] - 0s 254ms/step
69.43% completed!
1/1 [==============================] - 0s 258ms/step
69.53% completed!
1/1 [==============================] - 0s 263ms/step
69.63% completed!
1/1 [==============================] - 0s 261ms/step
69.73% completed!
1/1 [==============================] - 0s 353ms/step
69.83% completed!
1/1 [==============================] - 0s 265ms/step
69.93% completed!
1/1 [==============================] - 0s 295ms/step
70.03% completed!
1/1 [==============================] - 0s 268ms/step
70.13% completed!
1/1 [==============================] - 0s 272ms/step
70.23% completed!
1/1 [==============================] - 0s 316ms/step
70.33% completed!
1/1 [==============================] - 0s 283ms/step
70.43% completed!
1/1 [=

1/1 [==============================] - 0s 283ms/step
80.62% completed!
1/1 [==============================] - 0s 383ms/step
80.72% completed!
1/1 [==============================] - 0s 417ms/step
80.82% completed!
1/1 [==============================] - 0s 407ms/step
80.92% completed!
1/1 [==============================] - 0s 343ms/step
81.02% completed!
1/1 [==============================] - 0s 337ms/step
81.12% completed!
1/1 [==============================] - 0s 331ms/step
81.22% completed!
1/1 [==============================] - 0s 375ms/step
81.32% completed!
1/1 [==============================] - 0s 394ms/step
81.42% completed!
1/1 [==============================] - 0s 314ms/step
81.52% completed!
1/1 [==============================] - 0s 295ms/step
81.62% completed!
1/1 [==============================] - 0s 314ms/step
81.72% completed!
1/1 [==============================] - 0s 270ms/step
81.82% completed!
1/1 [==============================] - 0s 287ms/step
81.92% completed!
1/1 [=

1/1 [==============================] - 0s 267ms/step
92.11% completed!
1/1 [==============================] - 0s 267ms/step
92.21% completed!
1/1 [==============================] - 0s 410ms/step
92.31% completed!
1/1 [==============================] - 0s 266ms/step
92.41% completed!
1/1 [==============================] - 0s 264ms/step
92.51% completed!
1/1 [==============================] - 0s 254ms/step
92.61% completed!
1/1 [==============================] - 0s 252ms/step
92.71% completed!
1/1 [==============================] - 0s 251ms/step
92.81% completed!
1/1 [==============================] - 0s 267ms/step
92.91% completed!
1/1 [==============================] - 0s 282ms/step
93.01% completed!
1/1 [==============================] - 0s 267ms/step
93.11% completed!
1/1 [==============================] - 0s 283ms/step
93.21% completed!
1/1 [==============================] - 0s 284ms/step
93.31% completed!
1/1 [==============================] - 0s 292ms/step
93.41% completed!
1/1 [=

In [42]:
print(len(temp))
f.close()

1001


In [5]:
import cv2
import numpy as np
import glob
cap1 = cv2.VideoCapture("C://Users//Lohith//Downloads//RcbVsSrh - HD 720p.mp4")
no_of_frame1 = int(cap1.get(7))    
print(no_of_frame1)
img_array = []
i = 0
while i < no_of_frame1-2:
    try:
        ret,frame = cap1.read()
        frame = cv2.resize(frame,(720,480))
    except:
        continue
    value = temp[i]
    frame = cv2.putText(frame, value, (25, 25), cv2.FONT_ITALIC, 1, (0, 125, 255), 3,cv2.LINE_AA)
    if value=='Pitch':
        img_array.append(frame)
    
    i+=1


out = cv2.VideoWriter("C:/Users/Lohith/Desktop/Captsone Review 2/Sample_Trim6.mp4",cv2.VideoWriter_fourcc(*'DIVX'), 15, (720,480))
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

100


In [7]:
print((temp[422]))

['Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'No

In [7]:
cd dataset


C:\Users\Lohith\dataset


In [8]:
vidObj = cv2.VideoCapture("C:/Users/Lohith/Downloads/WhatsApp Video 2022-11-10 at 6.42.34 PM.mp4")
  
    # Used as counter variable
count = 0
  
    # checks whether frames were extracted
success = 1
  
while success:
  
        # vidObj object calls read
        # function extract frames
    success, image = vidObj.read()
  
        # Saves the frames with frame-count
    cv2.imwrite("frame%d.jpg" % count, image)
  
    count += 1

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [1]:
x=['Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch', 'Non-Pitch']

In [2]:
print(len(x))

1171


In [5]:
import matplotlib.pyplot as plt
import cv2
import easyocr
from pylab import rcParams
from IPython.display import Image
rcParams['figure.figsize'] = 8, 16
reader = easyocr.Reader(['en'])
import pandas as pd
import numpy as np
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
import pickle
with open('cric_shots.pkl', 'rb') as f:
    model = pickle.load(f)

In [10]:
c=0
battingteam=""
noofovers=""
score=""
bowler=""
strike=""
nonstrike=""
prev={}
img_array = []
cap1 = cv2.VideoCapture("C:/Users/Lohith/Downloads/Abd.mp4")

no_of_frame1 = int(cap1.get(7))    
print(no_of_frame1)
i = 0
while i < no_of_frame1-2:
    try:
        ret,frame = cap1.read()
    except:
        continue
    
    if x[i]=="Pitch":
        try:
            image1=frame
            image = cv2.cvtColor(frame[int(0.2*height):int(0.7*height)], cv2.COLOR_BGR2RGB)
        except:
            break
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #image=cv2.flip(image,1)
        
        
        
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
    
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            '''face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())'''
            
            # Concate rows
            row = pose_row
            
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class,body_language_prob)
    
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image1, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image1, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image1, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image1, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image1, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image1, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image1, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
    try:
        output = reader.readtext(image1)
    except:
        break
    if ret:
        teams=["RCB","SRH","CSK","MI","DD","KKR","RR","KXIP"]
# Displaying the image
        resulttxt=[]
        for i in output:
            resulttxt.append(i[1])
            
        for i in range(len(resulttxt)):
            if resulttxt[i] in teams:
                battingteam=resulttxt[i]
                try:
                    strike=resulttxt[i+1]
                except:
                    strike=""
            if resulttxt[i].startswith("OVERS"):
                noofovers=resulttxt[i]
                try:
                    nonstrike=resulttxt[i+1]
                except:
                    nonstrike=""
            if "-" in list(resulttxt[i]):
                score=resulttxt[i]
            if resulttxt[i]=="TIMELINE":
                try:
                    bowler=resulttxt[i+1]
                except:
                    bowler=""
        font = cv2.FONT_HERSHEY_SIMPLEX
        org = (500, 750)
        fontScale = 2
        color = (240,240,240)
        thickness = 5     
        if len(output)>14 and len(output)<19:
            d={"Batting Team "+battingteam:(50,400),noofovers:(50,475),"Score "+score:(50,550),bowler+" bowling to "+strike:(500,800),nonstrike+" in the other end ":(500,900)}
            prev=d
# Using cv2.putText() method
            for i in d:
                image = cv2.putText(image1,i, d[i], font,fontScale, color, thickness, cv2.LINE_AA)
        else:
            for i in prev:
                image = cv2.putText(image1,i, d[i], font,fontScale, color, thickness, cv2.LINE_AA)
  
    cv2.imshow("frame",image1)
    img_array.append(image1)
    
    i+=1
    
out = cv2.VideoWriter("C:/Users/Lohith/Desktop/Captsone Review 2/Sample_Trim6.mp4",cv2.VideoWriter_fourcc(*'DIVX'), 15, (720,480))
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

31


In [8]:
cd frame

C:\Users\Lohith\frame


In [9]:
cd

C:\Users\Lohith


In [35]:
cd C:\Users\Lohith\Capstone Review 3\Time Analysis

C:\Users\Lohith\Capstone Review 3\Time Analysis
